# Problem Statement
### Settling down in Toronto – Best place for an Indian based on number of Indian restaurants.
Toronto is a large city and it is not easy for an Indian migrating to Toronto to determine the neighbourhood he should choose to settle down as it has a lot of diversity in demographics and various other factors. Let us take the number of Indian restaurants in the neighbourhood to choose which neighbourhood would be best for him to settle down. 

We visualise all neighbourhoods of Toronto and also compare the number of Indian restaurants as a parameter to choose the best place to settle down.

# Data
### For this project we need the following data:
*Data source* : http://cocl.us/Geospatial_data

*Description* : We will use this data for Toronto City which contains a list of Neighbourhoods along with their latitude and longitude.

*Data source* : Fousquare API

*Description* : Using Foursquare api, we will get all the venues in each neighbourhood and then we will filter these venues for only Indian restaurants.
I will use the Kmeans 

# Solution methodology
From the above 2 data sources, I will be conducting clustering and neighborhood based analysis leveraging primarily Foursquare APIs and tools such as KNN to deliver recommendations options to the target user community primarily comprising of the Indian community.

In [1]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ca-certificates-2019.3.9   |       hecc5488_0         146 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

   

In [2]:
CLIENT_ID = '1UKCODX3PQXELJTD4TVKCTIQAXGVDZ3JPZXGTTEPZGGRTXLY' # your Foursquare ID
CLIENT_SECRET = 'NQJ2ZKJXTR1BYOIDLE3HZK1RH3NQNSWLMZSBUB3IN5ASJQ3E' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1UKCODX3PQXELJTD4TVKCTIQAXGVDZ3JPZXGTTEPZGGRTXLY
CLIENT_SECRET:NQJ2ZKJXTR1BYOIDLE3HZK1RH3NQNSWLMZSBUB3IN5ASJQ3E


In [3]:
address = 'Toronto, Ontario'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


43.653963 -79.387207


In [38]:
search_query = '"Indian Restaurant"'
radius = 10000
LIMIT=100

In [39]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
print(url)

https://api.foursquare.com/v2/venues/search?client_id=1UKCODX3PQXELJTD4TVKCTIQAXGVDZ3JPZXGTTEPZGGRTXLY&client_secret=NQJ2ZKJXTR1BYOIDLE3HZK1RH3NQNSWLMZSBUB3IN5ASJQ3E&ll=43.653963,-79.387207&v=20180605&query="Indian Restaurant"&radius=10000&limit=100


In [40]:
# Get the json output for the search query "Indian"
result = requests.get(url).json()  
result

{'meta': {'code': 200, 'requestId': '5cf1047a351e3d1286ada83a'},
 'response': {'venues': [{'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/indian_',
       'suffix': '.png'},
      'id': '4bf58dd8d48988d10f941735',
      'name': 'Indian Restaurant',
      'pluralName': 'Indian Restaurants',
      'primary': True,
      'shortName': 'Indian'}],
    'hasPerk': False,
    'id': '4aef8854f964a5201cd921e3',
    'location': {'address': '287 King St. W',
     'cc': 'CA',
     'city': 'Toronto',
     'country': 'Canada',
     'crossStreet': 'at John St.',
     'distance': 857,
     'formattedAddress': ['287 King St. W (at John St.)',
      'Toronto ON M5V 1J5',
      'Canada'],
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.646462521503445,
       'lng': -79.38964414801342}],
     'lat': 43.646462521503445,
     'lng': -79.38964414801342,
     'postalCode': 'M5V 1J5',
     'state': 'ON'},
    'name': 'Aroma Fine Indian Restaurant',
    'referralId

In [37]:
# assigning relevant part of JSON to venues
venues = result['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'pluralName': 'Indian Restaurants', 'id': '4...",False,4aef8854f964a5201cd921e3,287 King St. W,CA,Toronto,Canada,at John St.,857,"[287 King St. W (at John St.), Toronto ON M5V ...","[{'lat': 43.646462521503445, 'label': 'display...",43.646463,-79.389644,NaN,M5V 1J5,ON,Aroma Fine Indian Restaurant,v-1559298365,NaN
1,"[{'pluralName': 'Indian Restaurants', 'id': '4...",False,5165c333e4b07a7ad88d8a69,NaN,CA,NaN,Canada,NaN,651,[Canada],"[{'lat': 43.65814977325445, 'label': 'display'...",43.658150,-79.381563,NaN,NaN,NaN,Joe's Indian Restaurant,v-1559298365,NaN
2,"[{'pluralName': 'Fast Food Restaurants', 'id':...",False,53a07ba3498ee8946e98a7de,552 Mt Pleasant,CA,Toronto,Canada,NaN,1098,"[552 Mt Pleasant, Toronto ON M4S 2M6, Canada]","[{'lat': 43.64430171166487, 'label': 'display'...",43.644302,-79.390002,NaN,M4S 2M6,ON,Marigold Indian Bistro | Indian Restaurants in...,v-1559298365,NaN
3,"[{'pluralName': 'American Restaurants', 'id': ...",False,4ad4c05ff964a52048f720e3,110 Chestnut Street,CA,Toronto,Canada,NaN,145,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]","[{'lat': 43.65488413420439, 'label': 'display'...",43.654884,-79.385931,NaN,M5G 1R3,ON,Hemispheres Restaurant & Bistro,v-1559298365,NaN
4,"[{'pluralName': 'Indian Restaurants', 'id': '4...",False,4afd920ff964a520ad2822e3,181 Dundas St W,CA,Toronto,Canada,W of Chestnut St,136,"[181 Dundas St W (W of Chestnut St), Toronto O...","[{'lat': 43.65511996683289, 'label': 'display'...",43.655120,-79.386645,NaN,M5G 1C7,ON,Indian Biriyani House,v-1559298365,NaN


In [41]:
# Obtaining only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Aroma Fine Indian Restaurant,Indian Restaurant,287 King St. W,CA,Toronto,Canada,at John St.,857,"[287 King St. W (at John St.), Toronto ON M5V ...","[{'lat': 43.646462521503445, 'label': 'display...",43.646463,-79.389644,NaN,M5V 1J5,ON,4aef8854f964a5201cd921e3
1,Joe's Indian Restaurant,Indian Restaurant,NaN,CA,NaN,Canada,NaN,651,[Canada],"[{'lat': 43.65814977325445, 'label': 'display'...",43.658150,-79.381563,NaN,NaN,NaN,5165c333e4b07a7ad88d8a69
2,Marigold Indian Bistro | Indian Restaurants in...,Fast Food Restaurant,552 Mt Pleasant,CA,Toronto,Canada,NaN,1098,"[552 Mt Pleasant, Toronto ON M4S 2M6, Canada]","[{'lat': 43.64430171166487, 'label': 'display'...",43.644302,-79.390002,NaN,M4S 2M6,ON,53a07ba3498ee8946e98a7de
3,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,CA,Toronto,Canada,NaN,145,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]","[{'lat': 43.65488413420439, 'label': 'display'...",43.654884,-79.385931,NaN,M5G 1R3,ON,4ad4c05ff964a52048f720e3
4,Indian Biriyani House,Indian Restaurant,181 Dundas St W,CA,Toronto,Canada,W of Chestnut St,136,"[181 Dundas St W (W of Chestnut St), Toronto O...","[{'lat': 43.65511996683289, 'label': 'display'...",43.655120,-79.386645,NaN,M5G 1C7,ON,4afd920ff964a520ad2822e3
5,Indian Flavour,Indian Restaurant,123 Dundas St W,CA,Toronto,Canada,btw Elizabeth & Bay,311,"[123 Dundas St W (btw Elizabeth & Bay), Toront...","[{'lat': 43.65564910619165, 'label': 'display'...",43.655649,-79.384119,NaN,NaN,ON,4b2a634af964a52020a824e3
6,Victoria's Restaurant,Restaurant,37 King Street East,CA,Toronto,Canada,at Le Meridien King Edward Hotel,1011,[37 King Street East (at Le Meridien King Edwa...,"[{'lat': 43.64929834396347, 'label': 'display'...",43.649298,-79.376431,NaN,M5C 1E9,ON,4ad4c05cf964a52006f620e3
7,Hong Shing Chinese Restaurant,Chinese Restaurant,195 Dundas St W,CA,Toronto,Canada,at University Ave,107,"[195 Dundas St W (at University Ave), Toronto ...","[{'lat': 43.65492521335936, 'label': 'display'...",43.654925,-79.387089,NaN,M5G 1C7,ON,4b2027b5f964a520f82d24e3
8,Indian Biriyani House,Indian Restaurant,120 Adelaide Street West,CA,Toronto,Canada,at Bay Street,683,"[120 Adelaide Street West (at Bay Street), Tor...","[{'lat': 43.650050280518336, 'label': 'display...",43.650050,-79.380662,NaN,M5H 3V1,ON,4fea497c121d2480d3579412
9,Indian Road Crescent Public School,School,285 Indian Road Cres,CA,Toronto,Canada,NaN,5907,"[285 Indian Road Cres, Toronto ON M6P 2G8, Can...","[{'lat': 43.66249140470172, 'label': 'display'...",43.662491,-79.459608,NaN,M6P 2G8,ON,4cc5b3babe40a35d7c01864c


In [42]:
dataframe_filtered.describe()

,distance,lat,lng
count,50.000000,50.000000,50.000000
mean,1009.320000,43.652627,-79.390431
std,929.557069,0.006423,0.014168
min,96.000000,43.639060,-79.459608
25%,544.000000,43.648168,-79.393464
50%,890.500000,43.652766,-79.386510
75%,1086.500000,43.655647,-79.382246
max,5907.000000,43.670574,-79.374814


In [51]:
dataframe_cleaned = dataframe_filtered[dataframe_filtered['address'].notnull()]  # get rid records with address "Not available" 
dataframe_TOR=dataframe_cleaned[dataframe_cleaned.state == 'ON']   # get rid of "non ON" states
df_withpostcode=dataframe_TOR[dataframe_TOR['postalCode'].notnull()]  # get rid records with no Postcode
df_withpostcode


,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Aroma Fine Indian Restaurant,Indian Restaurant,287 King St. W,CA,Toronto,Canada,at John St.,857,"[287 King St. W (at John St.), Toronto ON M5V ...","[{'lat': 43.646462521503445, 'label': 'display...",43.646463,-79.389644,NaN,M5V 1J5,ON,4aef8854f964a5201cd921e3
2,Marigold Indian Bistro | Indian Restaurants in...,Fast Food Restaurant,552 Mt Pleasant,CA,Toronto,Canada,NaN,1098,"[552 Mt Pleasant, Toronto ON M4S 2M6, Canada]","[{'lat': 43.64430171166487, 'label': 'display'...",43.644302,-79.390002,NaN,M4S 2M6,ON,53a07ba3498ee8946e98a7de
3,Hemispheres Restaurant & Bistro,American Restaurant,110 Chestnut Street,CA,Toronto,Canada,NaN,145,"[110 Chestnut Street, Toronto ON M5G 1R3, Canada]","[{'lat': 43.65488413420439, 'label': 'display'...",43.654884,-79.385931,NaN,M5G 1R3,ON,4ad4c05ff964a52048f720e3
4,Indian Biriyani House,Indian Restaurant,181 Dundas St W,CA,Toronto,Canada,W of Chestnut St,136,"[181 Dundas St W (W of Chestnut St), Toronto O...","[{'lat': 43.65511996683289, 'label': 'display'...",43.655120,-79.386645,NaN,M5G 1C7,ON,4afd920ff964a520ad2822e3
6,Victoria's Restaurant,Restaurant,37 King Street East,CA,Toronto,Canada,at Le Meridien King Edward Hotel,1011,[37 King Street East (at Le Meridien King Edwa...,"[{'lat': 43.64929834396347, 'label': 'display'...",43.649298,-79.376431,NaN,M5C 1E9,ON,4ad4c05cf964a52006f620e3
7,Hong Shing Chinese Restaurant,Chinese Restaurant,195 Dundas St W,CA,Toronto,Canada,at University Ave,107,"[195 Dundas St W (at University Ave), Toronto ...","[{'lat': 43.65492521335936, 'label': 'display'...",43.654925,-79.387089,NaN,M5G 1C7,ON,4b2027b5f964a520f82d24e3
8,Indian Biriyani House,Indian Restaurant,120 Adelaide Street West,CA,Toronto,Canada,at Bay Street,683,"[120 Adelaide Street West (at Bay Street), Tor...","[{'lat': 43.650050280518336, 'label': 'display...",43.650050,-79.380662,NaN,M5H 3V1,ON,4fea497c121d2480d3579412
9,Indian Road Crescent Public School,School,285 Indian Road Cres,CA,Toronto,Canada,NaN,5907,"[285 Indian Road Cres, Toronto ON M6P 2G8, Can...","[{'lat': 43.66249140470172, 'label': 'display'...",43.662491,-79.459608,NaN,M6P 2G8,ON,4cc5b3babe40a35d7c01864c
10,Green Tea Restaurant Downtown,Restaurant,261 Spadina Avenue. Upper level,CA,Toronto,Canada,NaN,845,"[261 Spadina Avenue. Upper level, Toronto ON M...","[{'lat': 43.652487830335545, 'label': 'display...",43.652488,-79.397501,NaN,M5T 2E3,ON,5c33c7cd8194fc002c66f72e
11,Azure Restaurant & Bar,Restaurant,225 Frnt St W,CA,Toronto,Canada,NaN,1039,"[225 Frnt St W, Toronto ON M5V 2X3, Canada]","[{'lat': 43.644749195919346, 'label': 'display...",43.644749,-79.385113,Entertainment District,M5V 2X3,ON,4b223f5af964a520ba4424e3


In [52]:
# define the dataframe columns
column_names = ['postalcode', 'Latitude', 'Longitude'] 

# instantiate the dataframe
df_postcode = pd.DataFrame(columns=column_names)
df_postcode

,postalcode,Latitude,Longitude


In [126]:
#Filling data for each row
df_postcode = df_withpostcode[['postalCode','lat','lng']]
df_postcode.head()

,postalCode,lat,lng
0,M5V 1J5,43.646463,-79.389644
2,M4S 2M6,43.644302,-79.390002
3,M5G 1R3,43.654884,-79.385931
4,M5G 1C7,43.655120,-79.386645
6,M5C 1E9,43.649298,-79.376431


In [87]:
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto City are 43.653963, -79.387207.


In [114]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, postalCode in zip(df_postcode['lat'], df_postcode['lng'], df_postcode['postalCode']):
    label = '{}'.format(postalCode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [152]:
#Filling data for each row
df_postcode = df_withpostcode[['postalCode','lat','lng']]
df_postcode.head()

,postalCode,lat,lng
0,M5V 1J5,43.646463,-79.389644
2,M4S 2M6,43.644302,-79.390002
3,M5G 1R3,43.654884,-79.385931
4,M5G 1C7,43.655120,-79.386645
6,M5C 1E9,43.649298,-79.376431


In [153]:
df_postcode['postalCode']=df_postcode['postalCode'].str[:3]
df_postcode.head()

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,postalCode,lat,lng
0,M5V,43.646463,-79.389644
2,M4S,43.644302,-79.390002
3,M5G,43.654884,-79.385931
4,M5G,43.655120,-79.386645
6,M5C,43.649298,-79.376431


In [154]:
df_postcode=df_postcode.groupby('postalCode').count() 
df_postcode

,lat,lng
postalCode,,
M4S,1,1
M4W,1,1
M4Y,1,1
M5B,2,2
M5C,2,2
M5E,1,1
M5G,6,6
M5H,3,3
M5J,2,2


Highest count of Indian Restaurants are in M5G (6), M5T (5) and M5V (5)

We can see that postal code starting with M5G is the best place. Second best place for an indian would be M5T and M5V. 
Quick search in wikipedia "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" indicates that 

for M5G Borough is "DownTown Toronto"
for M5T Borough is "DownTown Toronto"
for M5V Borough is "DownTown Toronto"

With this we conculde that for any new Indian, it would be best to settle down near "DownTown Toronto" on the basis of the number of Indian Resturants. 